In [2]:
!pip install mlflow optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
import mlflow
import mlflow.sklearn
import optuna
from sklearn.metrics import mean_squared_error

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor

подключаем диск

In [4]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive

Mounted at /content/drive/
/content/drive/MyDrive


Скачайте датасет, выберите столбец Price в качестве целевого, удалите его из X, затем сделайте сплит.

In [11]:
df = pd.read_csv('/content/drive/MyDrive/fourthsemestr/pipeline/Laptop_price.csv')
X = df.drop('Price', axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Создайте переменные для категориальных и количественных типов данных и запишите в них данные.

In [12]:
categoricalFeatures = X.select_dtypes(include=['object']).columns.tolist()
numericalFeatures = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

Далее создайте пайплайн

In [13]:
numericalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categoricalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numericalTransformer, numericalFeatures),
        ('cat', categoricalTransformer, categoricalFeatures)
    ])

In [15]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(random_state=42))
])

Использование Optuna для подбора гиперпараметров

In [18]:
def objective(trial):
    params = {
        'model__n_estimators': trial.suggest_int('model__n_estimators', 50, 200),
        'model__learning_rate': trial.suggest_float('model__learning_rate', 0.01, 0.2),
        'model__max_depth': trial.suggest_int('model__max_depth', 3, 7),
        'model__gamma': trial.suggest_float('model__gamma', 0, 0.2),
        'model__subsample': trial.suggest_float('model__subsample', 0.8, 1.0)
    }

    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, preds)

    return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

bestTrial = study.best_trial
bestParams = bestTrial.params

print("Лучшие параметры: ", bestParams)

[I 2024-06-03 23:33:36,757] A new study created in memory with name: no-name-dd818385-8688-4baf-8da3-ed90543ee303
[I 2024-06-03 23:33:36,852] Trial 0 finished with value: 5718008.891749412 and parameters: {'model__n_estimators': 91, 'model__learning_rate': 0.015248518887471998, 'model__max_depth': 7, 'model__gamma': 0.019342730197929425, 'model__subsample': 0.8987400425619204}. Best is trial 0 with value: 5718008.891749412.
[I 2024-06-03 23:33:36,990] Trial 1 finished with value: 108070.19176394094 and parameters: {'model__n_estimators': 104, 'model__learning_rate': 0.03435173203868138, 'model__max_depth': 6, 'model__gamma': 0.024308406589128053, 'model__subsample': 0.9094371482377219}. Best is trial 1 with value: 108070.19176394094.
[I 2024-06-03 23:33:37,797] Trial 2 finished with value: 44332.689355915805 and parameters: {'model__n_estimators': 166, 'model__learning_rate': 0.1769162845940855, 'model__max_depth': 7, 'model__gamma': 0.10109913667177764, 'model__subsample': 0.889454224

Лучшие параметры:  {'model__n_estimators': 174, 'model__learning_rate': 0.03414354581721668, 'model__max_depth': 5, 'model__gamma': 0.12625251224841255, 'model__subsample': 0.9164599567902671}


Применение этих параметров

In [21]:
pipeline.set_params(**bestParams)
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
mse = mean_squared_error(y_test, preds)

print(f"Mean Squared Error: {mse}")

Mean Squared Error: 38334.205029314544


In [23]:
mlflow.set_experiment("XGBRegressor Optimization with Optuna")

with mlflow.start_run():
    mlflow.log_params(bestParams)
    mlflow.log_metric("mse", mse)
    mlflow.sklearn.log_model(pipeline, "model")

print("Модель и результаты сохранены в MLflow")

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Модель и результаты сохранены в MLflow
